In [2]:
import requests
import pandas as pd

CITIESCODE = [3300100, 3300159, 3300209, 3300225, 3300233, 3300258, 3300308, 3300407, 3300456, 3300506, 3300605, 3300704, 3300803,
            3300902, 3300936, 3300951, 3301009, 3301108, 3301157, 3301207, 3301306, 3301405, 3301504, 3301603, 3301702, 3301801, 3301850,
            3301876, 3301900, 3302007, 3302056, 3302106, 3302205, 3302254, 3302270, 3302304, 3302403, 3302452, 3302502, 3302601, 3302700,
            3302809, 3302858, 3302908, 3303005, 3303104, 3303203, 3303302, 3303401, 3303500, 3303609, 3303708, 3303807, 3303856, 3303906,
            3303955, 3304003, 3304102, 3304110, 3304128, 3304144, 3304151, 3304201, 3304300, 3304409, 3304508, 3304524, 3304557, 3304607,
            3304706, 3304755, 3304805, 3304904, 3305000, 3305109, 3305133, 3305158, 3305208, 3305307, 3305406, 3305505, 3305554, 3305604,
            3305703, 3305752, 3305802, 3305901, 3306008, 3306107, 3306156, 3306206, 3306305]

In [3]:
# Emprego por Setor Econômico 2022
def getWorkTendency():
    data1 = []
    data2 = []
    data3 = []
    try:
        for cityCode in CITIESCODE:
            
            url = f"https://api-datampe.sebrae.com.br/data.jsonrecords?Municipality={cityCode}&cube=RAIS_establishment&drilldowns=Year,Division&measures=Number%20workers&parents=true&a=a"
            response = requests.get(url)
            if(response.status_code == 200):
                json_response = response.json()
                for obj in json_response["data"]:
                    if(obj["Year"] in [2019, 2020,2021]):
                        data1.append({"Year": obj["Year"], "MunicipalityID": cityCode, "GrandeSectorID": obj["Grande Sector ID"],
                                    "GrandeSetor": obj["Grande Sector"], "SectionID": obj["Section ID"],
                                    "Section": obj["Section"], "DivisionID": obj["Division ID"],
                                    "Division": obj["Division"], "NumberWorkers": obj["Number workers"]})
        
        for cityCode in CITIESCODE:
            url = f"https://api-datampe.sebrae.com.br/data.jsonrecords?Municipality={cityCode}&cube=RAIS_establishment&drilldowns=Year,Division&measures=Establishments&parents=true&a=a"
            response = requests.get(url)
            if(response.status_code == 200):
                json_response = response.json()
                for obj in json_response["data"]:
                    if(obj["Year"] in [2019, 2020,2021]):
                        data2.append({"Year": obj["Year"], "MunicipalityID": cityCode, "GrandeSectorID": obj["Grande Sector ID"],
                                    "GrandeSetor": obj["Grande Sector"], "SectionID": obj["Section ID"],
                                    "Section": obj["Section"], "DivisionID": obj["Division ID"],
                                    "Division": obj["Division"], "Establishments": obj["Establishments"]})

        for cityCode in CITIESCODE:
            url = f"https://api-datampe.sebrae.com.br/data.jsonrecords?Municipality={cityCode}&cube=RAIS_workers&drilldowns=Year,Division&measures=Remuneration%20Avg%20Nominal&parents=true&Active+worker+indicator=1"
            response = requests.get(url)
            if(response.status_code == 200):
                json_response = response.json()
                for obj in json_response["data"]:
                    if(obj["Year"] in [2019, 2020,2021]):
                        data3.append({"Year": obj["Year"], "MunicipalityID": cityCode, "GrandeSectorID": obj["Grande Sector ID"],
                                    "GrandeSetor": obj["Grande Sector"], "SectionID": obj["Section ID"],
                                    "Section": obj["Section"], "DivisionID": obj["Division ID"],
                                    "Division": obj["Division"], "RemunerationAvgNominal": obj["Remuneration Avg Nominal"]})
        

        df1 = pd.DataFrame(data1)
        df2 = pd.DataFrame(data2)
        df3 = pd.DataFrame(data3)

        merged_df1 = pd.merge(df1, df2, on=["Year", "MunicipalityID", "GrandeSectorID", "GrandeSetor", "SectionID", "Section", "DivisionID", "Division"], how="left")
        dfm1 = pd.DataFrame(merged_df1)
        merged_df2 = pd.merge(dfm1, df3, on=["Year", "MunicipalityID", "GrandeSectorID", "GrandeSetor", "SectionID", "Section", "DivisionID", "Division"], how="left")
        dfm2 = pd.DataFrame(merged_df2)
        dfm2["RemunerationAvgNominal"].fillna(0, inplace=True)

        dfm2.to_csv("../Data/workers_tendency.csv", index=False)

    except Exception as e:
        print(e)

getWorkTendency()

In [ ]:
# Numero de empregados 2022
def getTotalWorkers():
    data = []
    try:
        for cityCode in CITIESCODE:
            url = f"https://datampe.sebrae.com.br/proxy/data.jsonrecords?Grande+Sector=&Active+worker+indicator=1&Municipality={cityCode}&cube=RAIS_workers&drilldowns=Municipality,Year=2022&measures=Workers&q=4&parents=true"
            response = requests.get(url)
            if(response.status_code == 200):
                json_response = response.json()
                data.append({"MunicipalityID": json_response["data"][0]["Municipality ID"],
                             "Municipality": json_response["data"][0]["Municipality"],
                             "Workers": json_response["data"][0]["Workers"]})
                
            df = pd.DataFrame(data)
            df.to_csv("../Data/workers.csv", index=False)

    except Exception as e:
        print(e)


getTotalWorkers()

In [ ]:
# Diversidade de Emprego 2022
def getWorkersAgeDiversity():
    data = []
    try:
        for cityCode in CITIESCODE:
            url = f"https://datampe.sebrae.com.br/proxy/data.jsonrecords?Active+worker+indicator=1&Municipality={cityCode}&cube=RAIS_workers&drilldowns=Range%20Age,Sex,Year=2022&measures=Remuneration+Avg+Nominal,Workers&a=Workers&exclude=Range%20Age:99"
            response = requests.get(url)
            if(response.status_code == 200):
                json_response = response.json()
                for obj in json_response["data"]:
                    data.append({"MunicipalityID": cityCode, "RangeAge": obj["Range Age"],
                                "SexID": obj["Sex ID"], "Sex": obj["Sex"],
                                "RemunerationAvgNominal": obj["Remuneration Avg Nominal"], "Workers": obj["Workers"]})
                
            df = pd.DataFrame(data)
            df.to_csv("../Data/workers_age_diversity.csv", index=False)

    except Exception as e:
        print(e)


getWorkersAgeDiversity()

In [ ]:
# Índice de Mobilidade 2022
def getMobilityIndice():
    data = []
    try:
        for cityCode in CITIESCODE:
            url = f"https://datampe.sebrae.com.br/proxy/data.jsonrecords?Municipality={cityCode}&Grande+Sector=1%2C2%2C3%2C4%2C5&Time=2022&cube=CAGED_movements&drilldowns=Grande Sector&measures=Movement+Balance%2CAdmissions%2CResignations"
            response = requests.get(url)
            if(response.status_code == 200):
                json_response = response.json()
                for obj in json_response["data"]:
                    data.append({"MunicipalityID": cityCode, "GrandeSectorID": obj["Grande Sector ID"],
                                "GrandeSector": obj["Grande Sector"], "MovementBalance": obj["Movement Balance"],
                                "Admissions": obj["Admissions"], "Resignations": obj["Resignations"]})
                
            df = pd.DataFrame(data)
            df.to_csv("../Data/workers_mobility_indice.csv", index=False)

    except Exception as e:
        print(e)


getMobilityIndice()

In [ ]:
# Emprego por Tamanho da Empresa 2022
def getJobCompanySize():
    data = []
    try:
        for cityCode in CITIESCODE:
            url = f"https://datampe.sebrae.com.br/proxy/data.jsonrecords?Active+worker+indicator=1&Municipality={cityCode}&cube=RAIS_workers&drilldowns=Grande+Sector,Establishment+Size,Year=2022&measures=Remuneration+Avg+Nominal,Workers&a=Workers&exclude=Establishment+Size:0"
            response = requests.get(url)
            if(response.status_code == 200):
                json_response = response.json()
                for obj in json_response["data"]:
                    data.append({"MunicipalityID": cityCode, "GrandeSectorID": obj["Grande Sector ID"],
                                 "GrandeSector": obj["Grande Sector"], "EstablishmentSize": obj["Establishment Size"],
                                 "RemunerationAvgNominal": obj["Remuneration Avg Nominal"], "Workers": obj["Workers"]})
                
            df = pd.DataFrame(data)
            df.to_csv("../Data/job_by_company_size.csv", index=False)

    except Exception as e:
        print(e)

getJobCompanySize()

In [ ]:
# Evolução das Ocupações 2022
def getOcupationsEvolution():
    data = []
    try:
        for cityCode in CITIESCODE:
            url = f"https://datampe.sebrae.com.br/proxy/data?Active+worker+indicator=1&Municipality={cityCode}&cube=RAIS_workers&drilldowns=Subgroup,Active+worker+indicator,Year=2022&measures=Workers,Remuneration+Avg+Nominal&parents=true"
            response = requests.get(url)
            if(response.status_code == 200):
                json_response = response.json()
                for obj in json_response["data"]:
                    data.append({"MunicipalityID": cityCode, "CBOGroupID": obj["CBO Group ID"], "CBOGroup": obj["CBO Group"],
                                "SubgroupID": obj["Subgroup ID"],"Subgroup": obj["Subgroup"],
                                "ActiveWorkerIndicatorID": obj["Active worker indicator ID"], "ActiveWorkerIndicator": obj["Active worker indicator"],
                                "Workers": obj["Workers"], "RemunerationAvgNominal": obj["Remuneration Avg Nominal"]})
                
            df = pd.DataFrame(data)
            df.to_csv("../Data/ocupation_evolution.csv", index=False)

    except Exception as e:
        print(e)

getOcupationsEvolution()

In [ ]:
# Principais Ocupações 2022
def getMainOcupations():
    data = []
    try:
        for cityCode in CITIESCODE:
            url = f"https://datampe.sebrae.com.br/proxy/data?Active+worker+indicator=1&Municipality={cityCode}&cube=RAIS_workers&drilldowns=Occupation,Year=2022,Active+worker+indicator&measures=Workers&parents=true"
            response = requests.get(url)
            if(response.status_code == 200):
                json_response = response.json()
                for obj in json_response["data"]:
                    data.append({"MunicipalityID": cityCode, "CBOGroupID": obj["CBO Group ID"], "CBOGroup": obj["CBO Group"],
                                "SubgroupID": obj["Subgroup ID"],"Subgroup": obj["Subgroup"], "FamilyID": obj["Family ID"],
                                "Family": obj["Family"], "Occupation ID": obj["Occupation ID"],
                                "Workers": obj["Workers"]})
                
            df = pd.DataFrame(data)
            df.to_csv("../Data/main_ocupations.csv", index=False)

    except Exception as e:
        print(e)

getMainOcupations()

In [ ]:
# Distribuição de Ocupações 2022
# def getMainOcupations():
#     data = []
#     try:
#         for cityCode in CITIESCODE:
#             url = f"https://datampe.sebrae.com.br/proxy/data?Active+worker+indicator=1&Municipality={cityCode}&Year=2022&cube=RAIS_workers&drilldowns=Subgroup,Active+worker+indicator,Year&measures=Workers,Remuneration+Avg+Nominal&parents=true&Range+Age=&growth=Year,Workers"
#             response = requests.get(url)
#             if(response.status_code == 200):
#                 json_response = response.json()
#                 for obj in json_response["data"]:
#                     data.append({"MunicipalityID": cityCode, "CBOGroupID": obj["CBO Group ID"], "CBOGroup": obj["CBO Group"],
#                                 "SubgroupID": obj["Subgroup ID"],"Subgroup": obj["Subgroup"], "FamilyID": obj["Family ID"],
#                                 "Family": obj["Family"], "Occupation ID": obj["Occupation ID"],
#                                 "Workers": obj["Workers"], "ActiveWorkerIndicatorID": obj["Active worker indicator ID"],})
                
#             df = pd.DataFrame(data)
#             df.to_csv("../Data/main_ocupations.csv", index=False)

#     except Exception as e:
#         print(e)

# getMainOcupations()

In [ ]:
# Empresas por Porte e Natureza Jurídica + Tamanho 2022
def getCompaniesSize():
    data = []
    try:
        for cityCode in CITIESCODE:
            url = f"https://datampe.sebrae.com.br/proxy/data.jsonrecords?Grande+Sector=&Municipality={cityCode}&Year=2022&Registration+Status=2&cube=RF&drilldowns=Open+Activity+Year,Company+Size+Sebrae&measures=Establishments"
            response = requests.get(url)
            if(response.status_code == 200):
                json_response = response.json()
                for obj in json_response["data"]:
                    data.append({"MunicipalityID": cityCode, "OpenActivityYear": obj["Open Activity Year"],
                                 "CompanySizeSebraeID": obj["Company Size Sebrae ID"], "CompanySizeSebrae": obj["Company Size Sebrae"],
                                 "Establishments": obj["Establishments"]})
                
            df = pd.DataFrame(data)
            df.to_csv("../Data/companies.csv", index=False)

    except Exception as e:
        print(e)

getCompaniesSize()

In [ ]:
# Empresas por Atividade Principal 2022
def getCompaniesByMainActivity():
    data = []
    try:
        for cityCode in CITIESCODE:
            url = f"https://datampe.sebrae.com.br/proxy/data.jsonrecords?Registration+Status=2&Company+Size+Sebrae=&Municipality={cityCode}&Status+Year=2022&cube=RF&drilldowns=Status+Year,Division&measures=Establishments&parents=true&growth=Status+Year,Establishments"
            response = requests.get(url)
            if(response.status_code == 200):
                json_response = response.json()
                for obj in json_response["data"]:
                    data.append({"MunicipalityID": cityCode, "GrandeSectorID": obj["Grande Sector ID"], "GrandeSector": obj["Grande Sector"],
                                 "SectionID": obj["Section ID"], "Section": obj["Section"],
                                 "DivisionID": obj["Division ID"], "Division": obj["Division"],
                                 "Establishments": obj["Establishments"], "EstablishmentsGrowth": obj["Establishments Growth"],"EstablishmentsGrowthValue": obj["Establishments Growth Value"]})
                
            df = pd.DataFrame(data)
            df.to_csv("../Data/companies_by_activitiy.csv", index=False)

    except Exception as e:
        print(e)

getCompaniesByMainActivity()

In [ ]:
# População 2022
def getPopulation():
    data = []
    count = 0
    try:
        for cityCode in CITIESCODE:
            count += 1
            url = f"https://datampe.sebrae.com.br/proxy/data?cube=IBGE&drilldowns=Municipality,Year=2022&measures=Population&Municipality={cityCode}"
            response = requests.get(url)
            print(count)
            if(response.status_code == 200):
                json_response = response.json()
                for obj in json_response["data"]:
                    data.append({"MunicipalityID": cityCode, "Municipality": obj["Municipality"], "Population": obj["Population"]})
                
            df = pd.DataFrame(data)
            df.to_csv("../Data/population.csv", index=False)

    except Exception as e:
        print(e)

getPopulation()

In [ ]:
# Pontuação ENEM 2022
def getENEMPontuation():
    data = []
    try:
        for cityCode in CITIESCODE:
            url = f"https://datampe.sebrae.com.br/proxy/data.jsonrecords?Municipality={cityCode}&Year=2022&cube=INEP_enem&drilldowns=Sex&measures=Natural+Sciences,Social+Sciences,Languages,Math&exclude=Sex:0"
            response = requests.get(url)
            if(response.status_code == 200):
                json_response = response.json()
                for obj in json_response["data"]:
                    data.append({"MunicipalityID": cityCode, "SexID": obj["Sex ID"],
                                 "Sex": obj["Sex"], "NaturalSciences": obj["Natural Sciences"],
                                 "SocialSciences": obj["Social Sciences"], "Languages": obj["Languages"], "Math": obj["Math"]})
                
            df = pd.DataFrame(data)
            df.to_csv("../Data/enem_pontuation.csv", index=False)

    except Exception as e:
        print(e)

getENEMPontuation()

In [ ]:
# Pontuação ENEM ADMINISTRATION 2022
def getENEMPontuationAdministration():
    data = []
    try:
        for cityCode in CITIESCODE:
            url = f"https://datampe.sebrae.com.br/proxy/data.jsonrecords?Municipality={cityCode}&Year=2022&cube=INEP_enem&drilldowns=Administration&measures=Natural+Sciences,Social+Sciences,Languages,Math&exclude=Administration:0"
            response = requests.get(url)
            if(response.status_code == 200):
                json_response = response.json()
                for obj in json_response["data"]:
                    data.append({"MunicipalityID": cityCode, "AdministrationID": obj["Administration ID"],
                                 "Administration": obj["Administration"], "NaturalSciences": obj["Natural Sciences"],
                                 "SocialSciences": obj["Social Sciences"], "Languages": obj["Languages"], "Math": obj["Math"]})
                
            df = pd.DataFrame(data)
            df.to_csv("../Data/enem_pontuation_administration.csv", index=False)

    except Exception as e:
        print(e)

getENEMPontuationAdministration()

In [ ]:
# Educação superior 2022
def getCollege():
    data = []
    try:
        for cityCode in CITIESCODE:
            url = f"https://datampe.sebrae.com.br/proxy/data.jsonrecords?Year=2022&Municipality={cityCode}&cube=INEP_censo&drilldowns=OCDE+Specifies&measures=Enrollments&parents=true"
            response = requests.get(url)
            if(response.status_code == 200):
                json_response = response.json()
                for obj in json_response["data"]:
                    data.append({"MunicipalityID": cityCode, "OCDEGeneralID": obj["OCDE General ID"],"OCDEGeneral": obj["OCDE General"],
                                 "OCDESpecifiesID":obj["OCDE Specifies ID"],"OCDESpecifies": obj["OCDE Specifies"],"Enrollments":obj["Enrollments"]})
                
            df = pd.DataFrame(data)
            df.to_csv("../Data/college.csv", index=False)

    except Exception as e:
        print(e)

getCollege()

In [ ]:
# Alunos por gênero e grau acadêmico 2022
def getCollegeStudents():
    data = []
    try:
        for cityCode in CITIESCODE:
            url = f"https://datampe.sebrae.com.br/proxy/data?Municipality={cityCode}&cube=INEP_censo&drilldowns=Degree,Sex,Year=2022&measures=Enrollments&Administrative+Category=1,2,3,4,5"
            response = requests.get(url)
            if(response.status_code == 200):
                json_response = response.json()
                for obj in json_response["data"]:
                    data.append({"MunicipalityID": cityCode, "DegreeID": obj["Degree ID"],"SexID": obj["Sex ID"],
                                 "Sex":obj["Sex"],"Enrollments": obj["Enrollments"]})
                
            df = pd.DataFrame(data)
            df.to_csv("../Data/college_students.csv", index=False)

    except Exception as e:
        print(e)

getCollegeStudents()

In [ ]:
# Exportacao LKG 2022
def getExportacaoLKG():
    data = []
    try:
        for cityCode in CITIESCODE:
            url = f"https://datampe.sebrae.com.br/proxy/data.jsonrecords?Municipality={cityCode}&Codigo+SH2=&Operation+Type+ID=1&Year=2022&cube=mdic_exp_imp_mun&drilldowns=COMEX+-+Country+Code&measures=Liquid+Weight+KG"
            response = requests.get(url)
            if(response.status_code == 200):
                json_response = response.json()
                for obj in json_response["data"]:
                    data.append({"MunicipalityID": cityCode, "COMEXCountryCodeID": obj["COMEX - Country Code ID"],
                                 "COMEXCountryCode": obj["COMEX - Country Code"], "LiquidWeightKG": obj["Liquid Weight KG"]})
                
            df = pd.DataFrame(data)
            df.to_csv("../Data/exportacao_lkg.csv", index=False)

    except Exception as e:
        print(e)

getExportacaoLKG()

In [ ]:
# Exportacao DOLLAR 2022
def getExportacaoDollar():
    data = []
    try:
        for cityCode in CITIESCODE:
            url = f"https://datampe.sebrae.com.br/proxy/data.jsonrecords?Municipality={cityCode}&Codigo+SH2=&Operation+Type+ID=1&Year=2022&cube=mdic_exp_imp_mun&drilldowns=COMEX+-+Country+Code&measures=Dollar+Value+FOB"
            response = requests.get(url)
            if(response.status_code == 200):
                json_response = response.json()
                for obj in json_response["data"]:
                    data.append({"MunicipalityID": cityCode, "COMEXCountryCodeID": obj["COMEX - Country Code ID"],
                                 "COMEXCountryCode": obj["COMEX - Country Code"],"DollarValueFOB": obj["Dollar Value FOB"]})
                
            df = pd.DataFrame(data)
            df.to_csv("../Data/exportacao_dollar.csv", index=False)

    except Exception as e:
        print(e)

getExportacaoDollar()

In [ ]:
# Importacao DOLLAR 2022
def getExportacaoDollar():
    data = []
    try:
        for cityCode in CITIESCODE:
            url = f"https://datampe.sebrae.com.br/proxy/data.jsonrecords?Municipality={cityCode}&Codigo+SH2=&Operation+Type+ID=2&Year=2022&cube=mdic_exp_imp_mun&drilldowns=COMEX+-+Country+Code&measures=Dollar+Value+FOB"
            response = requests.get(url)
            if(response.status_code == 200):
                json_response = response.json()
                for obj in json_response["data"]:
                    data.append({"MunicipalityID": cityCode, "COMEXCountryCodeID": obj["COMEX - Country Code ID"],
                                 "COMEXCountryCode": obj["COMEX - Country Code"],"DollarValueFOB": obj["Dollar Value FOB"]})
                
            df = pd.DataFrame(data)
            df.to_csv("../Data/importacao_dollar.csv", index=False)

    except Exception as e:
        print(e)

getExportacaoDollar()

In [ ]:
# Importacao LKG 2022
def getExportacaoDollar():
    data = []
    try:
        for cityCode in CITIESCODE:
            url = f"https://datampe.sebrae.com.br/proxy/data.jsonrecords?Municipality={cityCode}&Codigo+SH2=&Operation+Type+ID=2&Year=2022&cube=mdic_exp_imp_mun&drilldowns=COMEX+-+Country+Code&measures=Liquid+Weight+KG"
            response = requests.get(url)
            if(response.status_code == 200):
                json_response = response.json()
                for obj in json_response["data"]:
                    data.append({"MunicipalityID": cityCode, "COMEXCountryCodeID": obj["COMEX - Country Code ID"],
                                 "COMEXCountryCode": obj["COMEX - Country Code"], "LiquidWeightKG": obj["Liquid Weight KG"]})
                
            df = pd.DataFrame(data)
            df.to_csv("../Data/importacao_lkg.csv", index=False)

    except Exception as e:
        print(e)

getExportacaoDollar()